In [1]:
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets.dataset_dict import DatasetDict
from datasets import Dataset

import utils.dataset_processors as dataset_processors
import numpy as np
import pandas as pd
import re

c:\Users\User\anaconda3\envs\ml_projects\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Initialize the model name for tokenizer and also the saved model name
model_name = 'distilbert-base-uncased'
saved_model = 'distillbert-finetuned-segmented'

In [3]:
# Load the dataset
datafile = "data/essays/essays.csv"
dataset = dataset_processors.load_essays_df(datafile)

# Split the dataset (6:2:2)
train_data, temp_data = train_test_split(dataset, train_size=0.6, random_state=42)
validation_data, test_data = train_test_split(temp_data, train_size=0.5, random_state=42)

In [4]:
# Get the Big 5 labels
column_names = list(train_data.columns)
labels = [label for label in column_names if label not in ['user','text','token_len']]

# Forward and backward mapping
id2label = {idx:label for idx,label in enumerate(labels)}
label2id = {label:idx for idx,label in enumerate(labels)}

labels

['EXT', 'NEU', 'AGR', 'CON', 'OPN']

In [5]:
# Import the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    problem_type = "multi_label_classification",
    num_labels = len(labels),
    id2label = id2label,
    label2id = label2id
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
# Convert from essay to sentences
def split_text_with_labels(row):
    
    # Split sentences
    sentences = re.split(r"(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?|\!)\s", row['text'])

    return [{
        'text': sentence,
        'EXT': row['EXT'],
        'NEU': row['NEU'],
        'AGR': row['AGR'],
        'CON': row['CON'],
        'OPN': row['OPN']
    }
        for sentence in sentences       
    ]

def transform_dataframe(old_dataframe):

    # Begin the split
    split_data = []
    
    for _, row in old_dataframe.iterrows():
        split_data.extend(split_text_with_labels(row))
        
    return pd.DataFrame(split_data)

train_data = transform_dataframe(train_data)
test_data = transform_dataframe(test_data)
validation_data = transform_dataframe(validation_data)

In [7]:
print(train_data.head(20))

                                                 text  EXT  NEU  AGR  CON  OPN
0                                     I am tired now.    1    0    1    0    1
1              I don't know what I should talk about.    1    0    1    0    1
2                             I like this assignment.    1    0    1    0    1
3                               Wonder when it's due?    1    0    1    0    1
4    Kristi Urey is the most beautiful person I know.    1    0    1    0    1
5                                         I love you.    1    0    1    0    1
6   I mean I love her with all of my heart, mind, ...    1    0    1    0    1
7                   I like psychology in high school.    1    0    1    0    1
8               It was very interesting and personal.    1    0    1    0    1
9                 That draws people in or so I think.    1    0    1    0    1
10                 Psychology is the food of the sea.    1    0    1    0    1
11                            My roommate's a weirdo

In [8]:
# Convert to DatasetDict
train_dataset = Dataset.from_dict(train_data)
test_dataset = Dataset.from_dict(test_data)
valid_dataset = Dataset.from_dict(validation_data)

full_dataset_dict = DatasetDict({
    "train": train_dataset,
    "test": test_dataset,
    "valid": valid_dataset
})

print(full_dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['text', 'EXT', 'NEU', 'AGR', 'CON', 'OPN'],
        num_rows: 73537
    })
    test: Dataset({
        features: ['text', 'EXT', 'NEU', 'AGR', 'CON', 'OPN'],
        num_rows: 25629
    })
    valid: Dataset({
        features: ['text', 'EXT', 'NEU', 'AGR', 'CON', 'OPN'],
        num_rows: 23811
    })
})


In [9]:
def preprocess_text(row):

    # Extract the text
    essays = row['text']

    # Clean up
    essays = [dataset_processors.preprocess_text(essay) for essay in essays]

    # Encode them using the tokenizer
    encoded_essay = tokenizer(essays, truncation=True)
    
    # Add the labels
    labels_batch = {key: row[key] for key in row.keys() if key in labels}

    # Create numpy array of batch and labels
    labels_matrix = np.zeros((len(essays), len(labels)))

    # Fill the array
    for idx, label in enumerate(labels):
        labels_matrix[:, idx] = labels_batch[label]

    # Return the encoding
    encoded_essay["labels"] = labels_matrix.tolist()
    
    return encoded_essay

# Perform the preprocessing
full_dataset_dict = full_dataset_dict.map(
    preprocess_text, batched = True, 
    remove_columns = full_dataset_dict['train'].column_names
)

Map: 100%|██████████| 23811/23811 [00:03<00:00, 6663.40 examples/s]


In [10]:
#print(full_dataset_dict['train'][0].keys())
print(full_dataset_dict['train'][0])
#print(full_dataset_dict['train'][0]['labels'])
#tokenizer.decode(full_dataset_dict['train'][5]['input_ids'])
#[id2label[idx] for idx, label in enumerate(full_dataset_dict['train'][5]['labels']) if label == 1.0]

{'input_ids': [101, 1045, 2572, 5458, 2085, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1], 'labels': [1.0, 0.0, 1.0, 0.0, 1.0]}


In [ ]:
full_dataset_dict.set_format("torch", device="cuda:0")
model = model.to("cuda:0")

In [12]:
from transformers import TrainingArguments, Trainer

batch_size = 16
learning_rate = 2e-5
epochs = 10
metric_name = "accuracy"

print(learning_rate)

2e-05


In [13]:
args = TrainingArguments(
    saved_model,
    evaluation_strategy = "epoch",
    save_strategy = "no",
    learning_rate = learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs = epochs,
    weight_decay = 0.01,
    metric_for_best_model = metric_name
)

c:\Users\User\anaconda3\envs\ml_projects\lib\site-packages\transformers\training_args.py:1545: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [14]:
print(labels)

['EXT', 'NEU', 'AGR', 'CON', 'OPN']


In [15]:
from sklearn.metrics import accuracy_score
import torch

def multi_label_metrics(pred_logits, gold_labels):

    # Our threshold
    threshold = 0.5

    # Apply sigmoid to logits
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(pred_logits))

    # Convert predictions to integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1

    # Compute metrics
    y_true = gold_labels

    metrics = {
        f"{id2label[i]} - accuracy": accuracy_score(y_true[:, i], y_pred[:, i]) 
        for i in range(len(labels))
    }

    overall_accuracy = accuracy_score(y_true, y_pred)

    # Store and return as dictionary
    metrics['accuracy'] = overall_accuracy
    
    return metrics

def compute_metrics(p):

    # Get the type of predictions
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions

    # Compute the results
    results = multi_label_metrics(preds, p.label_ids)

    return results

In [16]:
# forward pass (testing)
outputs = model(
    input_ids=full_dataset_dict['train']['input_ids'][0].unsqueeze(0), 
    attention_mask=full_dataset_dict['train']['attention_mask'][0].unsqueeze(0),
    labels=full_dataset_dict['train'][0]['labels'].unsqueeze(0)
)

outputs

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [ ]:
# Initialize the trainer before training
trainer = Trainer(
    model,
    args,
    train_dataset=full_dataset_dict["train"],
    eval_dataset=full_dataset_dict["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

  1%|          | 502/45970 [00:47<1:03:40, 11.90it/s]

{'loss': 0.691, 'grad_norm': 0.6273068189620972, 'learning_rate': 1.9782466826190996e-05, 'epoch': 0.11}


  2%|▏         | 1001/45970 [01:37<1:04:23, 11.64it/s]

{'loss': 0.6874, 'grad_norm': 0.5319086909294128, 'learning_rate': 1.956493365238199e-05, 'epoch': 0.22}


  3%|▎         | 1502/45970 [02:26<58:59, 12.56it/s]  

...

 99%|█████████▉| 45503/45970 [1:19:38<00:36, 12.95it/s]

{'loss': 0.3586, 'grad_norm': 6.320346832275391, 'learning_rate': 2.0448118338046557e-07, 'epoch': 9.9}


100%|██████████| 45970/45970 [1:21:05<00:00,  9.45it/s]

{'eval_loss': 1.169968605041504, 'eval_EXT - accuracy': 0.5155878106832105, 'eval_NEU - accuracy': 0.5146123531936478, 'eval_AGR - accuracy': 0.5128175114128526, 'eval_CON - accuracy': 0.5185532014514808, 'eval_OPN - accuracy': 0.5367357290569277, 'eval_accuracy': 0.04448086152405478, 'eval_runtime': 40.0307, 'eval_samples_per_second': 640.234, 'eval_steps_per_second': 40.019, 'epoch': 10.0}
{'train_runtime': 4865.7421, 'train_samples_per_second': 151.132, 'train_steps_per_second': 9.448, 'train_loss': 0.5153251616415937, 'epoch': 10.0}


TrainOutput(global_step=45970, training_loss=0.5153251616415937, metrics={'train_runtime': 4865.7421, 'train_samples_per_second': 151.132, 'train_steps_per_second': 9.448, 'total_flos': 9631412851951230.0, 'train_loss': 0.5153251616415937, 'epoch': 10.0})

In [ ]:
trainer.save_model('fine-tuned-roberta-personality-normal')